In [1]:
import numpy as np
from loguru import logger

from portfolioqtopt.optimization.markovitz_portfolio import Selection
from portfolioqtopt.optimization.optimizer import Optimizer, SolverTypes
from portfolioqtopt.optimization.qubo import QuboFactory
from portfolioqtopt.reader import read_welzia_stocks_file

file_path = "/home/ggelabert/Projects/PortfolioQtOpt/data/Histórico_carteras_Welzia_2018.xlsm"
sheet_name = "BBG (valores)"

df = read_welzia_stocks_file(file_path, sheet_name)
logger.info(f"{df.shape=}")
w = 5
b = 1.0
selection = Selection(df.to_numpy(), w, b)
qubo_factory = QuboFactory(selection, 0.9, 0.4, 0.1)

2023-01-20 11:55:59.670 | INFO     | __main__:<module>:13 - df.shape=(375, 45)


In [2]:
from portfolioqtopt.optimization.utils import get_partitions_granularity
w = 5
b = 1.0

In [3]:
pw = get_partitions_granularity(w)

In [1]:
import numpy as np
prices = np.array([[100, 50, 10, 5], [10, 5, 1, 0.5]]).T
n, m = prices.shape
p = m * w
factor = 1 / prices[-1, :]
normalized_prices = prices * factor
_npp = (normalized_prices[..., None] * pw)  # (n, m, w)
npp = _npp.reshape(n, -1).shape  # (n, p)

In [5]:
normalized_prices.shape

(4, 2)

In [6]:

# broadcast to m
import numpy as np

def get_extend_partitions_granularity(pw, m):
    pw_extend = np.zeros((m, 1)) + pw[None, :]
    return pw_extend.flatten()

In [7]:
%%timeit
m = 4
ed_pw = get_extend_partitions_granularity(pw, m)

2.8 µs ± 325 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [8]:
m = 4
ed_pw = get_extend_partitions_granularity(pw, m)
ed_pw

array([1.    , 0.5   , 0.25  , 0.125 , 0.0625, 1.    , 0.5   , 0.25  ,
       0.125 , 0.0625, 1.    , 0.5   , 0.25  , 0.125 , 0.0625, 1.    ,
       0.5   , 0.25  , 0.125 , 0.0625])

In [9]:
s = Selection(prices, w, b)
s.npp.shape

(4, 10)

In [10]:
average_daily_returns = (np.diff(prices, axis=0) / prices[:-1]).mean(axis=0)  # (m, )
(average_daily_returns[:, None] * pw[None, :]).flatten()

array([-0.6   , -0.3   , -0.15  , -0.075 , -0.0375, -0.6   , -0.3   ,
       -0.15  , -0.075 , -0.0375])

In [11]:
(prices[1:] - prices[:-1]).mean(axis=0), np.diff(prices, axis=0)

(array([-31.66666667,  -3.16666667]),
 array([[-50. ,  -5. ],
        [-40. ,  -4. ],
        [ -5. ,  -0.5]]))

In [12]:
s._expected_returns

array([-0.6   , -0.3   , -0.15  , -0.075 , -0.0375, -0.6   , -0.3   ,
       -0.15  , -0.075 , -0.0375])

In [1]:
from portfolioqtopt.optimization.utils import *
from portfolioqtopt.optimization.markovitz_portfolio import Selection
from portfolioqtopt.optimization.qubo import QuboFactory
import numpy as np

w = 5
prices = np.array([[100, 50, 10, 5], [10, 5, 1, 0.5]]).T
n, m = prices.shape
p = m * w
b = 1
selection = Selection(prices, w, b)
qubo = QuboFactory(selection, 0.1, 0.1, 0.1)

pw = get_partitions_granularity(w)

np_ = get_normalized_prices(prices, b)
npp = get_normalized_prices_partition(np_, pw)
np.testing.assert_equal(selection.npp, npp)

In [2]:
qubo = get_qubo(prices, b, w,  0.1, 0.1, 0.1)
qubo

{(0, 0): 10.825,
 (0, 1): 7.8583333333333325,
 (0, 2): 3.9291666666666663,
 (0, 3): 1.9645833333333331,
 (0, 4): 0.9822916666666666,
 (0, 5): 15.716666666666665,
 (0, 6): 7.8583333333333325,
 (0, 7): 3.9291666666666663,
 (0, 8): 1.9645833333333331,
 (0, 9): 0.9822916666666666,
 (1, 0): 0.0,
 (1, 1): 3.4479166666666665,
 (1, 2): 1.9645833333333331,
 (1, 3): 0.9822916666666666,
 (1, 4): 0.4911458333333333,
 (1, 5): 7.8583333333333325,
 (1, 6): 3.9291666666666663,
 (1, 7): 1.9645833333333331,
 (1, 8): 0.9822916666666666,
 (1, 9): 0.4911458333333333,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 1.2328125,
 (2, 3): 0.4911458333333333,
 (2, 4): 0.24557291666666664,
 (2, 5): 3.9291666666666663,
 (2, 6): 1.9645833333333331,
 (2, 7): 0.9822916666666666,
 (2, 8): 0.4911458333333333,
 (2, 9): 0.24557291666666664,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.4936197916666667,
 (3, 4): 0.12278645833333332,
 (3, 5): 1.9645833333333331,
 (3, 6): 0.9822916666666666,
 (3, 7): 0.4911458333333333,
 (3, 8

In [41]:
qubo.qubo.dictionary

{(0, 0): 8.291666666666666,
 (0, 1): 7.8583333333333325,
 (0, 2): 3.9291666666666663,
 (0, 3): 1.9645833333333331,
 (0, 4): 0.9822916666666666,
 (0, 5): 15.716666666666665,
 (0, 6): 7.8583333333333325,
 (0, 7): 3.9291666666666663,
 (0, 8): 1.9645833333333331,
 (0, 9): 0.9822916666666666,
 (1, 0): 0.0,
 (1, 1): 2.18125,
 (1, 2): 1.9645833333333331,
 (1, 3): 0.9822916666666666,
 (1, 4): 0.4911458333333333,
 (1, 5): 7.8583333333333325,
 (1, 6): 3.9291666666666663,
 (1, 7): 1.9645833333333331,
 (1, 8): 0.9822916666666666,
 (1, 9): 0.4911458333333333,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 0.5994791666666667,
 (2, 3): 0.4911458333333333,
 (2, 4): 0.24557291666666664,
 (2, 5): 3.9291666666666663,
 (2, 6): 1.9645833333333331,
 (2, 7): 0.9822916666666666,
 (2, 8): 0.4911458333333333,
 (2, 9): 0.24557291666666664,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.176953125,
 (3, 4): 0.12278645833333332,
 (3, 5): 1.9645833333333331,
 (3, 6): 0.9822916666666666,
 (3, 7): 0.4911458333333333,
 (3,

In [44]:
for k, v in zip(qubo_dict.items(), qubo.qubo.dictionary.items()):
    if k != v:
        print(k, v)

((0, 0), 10.825) ((0, 0), 8.291666666666666)
((1, 1), 3.4479166666666665) ((1, 1), 2.18125)
((2, 2), 1.2328125) ((2, 2), 0.5994791666666667)
((3, 3), 0.4936197916666667) ((3, 3), 0.176953125)
((4, 4), 0.21611328125) ((4, 4), 0.05777994791666667)
((5, 5), 7.974999999999999) ((5, 5), 8.291666666666666)
((6, 6), 2.0229166666666663) ((6, 6), 2.18125)
((7, 7), 0.5203125) ((7, 7), 0.5994791666666666)
((8, 8), 0.13736979166666666) ((8, 8), 0.176953125)
((9, 9), 0.03798828124999999) ((9, 9), 0.05777994791666666)


In [14]:
%%timeit
npp = get_normalized_prices_partition(np_, pw)

3.34 µs ± 447 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
%%timeit
pw = get_partitions_granularity(w)
np_ = get_normalized_prices(prices, b)

3.77 µs ± 563 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [2]:
a_ = get_average_daily_returns(prices)
a_pw = get_average_daily_returns_partition(a_, pw)
a_pw_ = selection._expected_returns
np.testing.assert_equal(a_pw, a_pw_)

In [17]:
%%timeit
a_ = get_average_daily_returns(prices)
a_pw = get_average_daily_returns_partition(a_, pw)

21.6 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
%%timeit
a_pw_ = selection._expected_returns

48.8 ns ± 3.87 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [3]:
selection.npp_last, pw

(array([1.    , 0.5   , 0.25  , 0.125 , 0.0625, 1.    , 0.5   , 0.25  ,
        0.125 , 0.0625]),
 array([1.    , 0.5   , 0.25  , 0.125 , 0.0625]))

In [4]:
(np.zeros((m, 1)) + pw).flatten()

array([1.    , 0.5   , 0.25  , 0.125 , 0.0625, 1.    , 0.5   , 0.25  ,
       0.125 , 0.0625])

In [33]:
np.testing.assert_equal(np.diag(selection._expected_returns), np.diag(a_pw))

In [3]:
def get_qubo(prices, b, w):
    # Compute the granularity partition
    pw = get_partitions_granularity(w)

    # Compute the  partition of the normalized prices
    np_ = get_normalized_prices(prices, b)
    npp = get_normalized_prices_partition(np_, pw)

    # Compute the partitions of the average daily returns.
    adr = get_average_daily_returns(prices)
    adrp = get_average_daily_returns_partition(a_, pw)

In [4]:
get_partitions_granularity_broadcast(pw, 2)
pwb = np.zeros((m, 1)) + pw  # (m, p)
pwb.flatten()

array([1.    , 0.5   , 0.25  , 0.125 , 0.0625, 1.    , 0.5   , 0.25  ,
       0.125 , 0.0625])

In [7]:
selection.expected_returns

array([-6.33333333, -3.16666667, -1.58333333, -0.79166667, -0.39583333,
       -6.33333333, -3.16666667, -1.58333333, -0.79166667, -0.39583333])

In [13]:
approximate_average = (prices[-1, :] - prices[0, :]) / (n - 1)  # average of the normalized prices?
factor = (b / prices[-1]).reshape(-1, 1)
(approximate_average[:, np.newaxis] * pw * factor).flatten()

array([-6.33333333, -3.16666667, -1.58333333, -0.79166667, -0.39583333,
       -6.33333333, -3.16666667, -1.58333333, -0.79166667, -0.39583333])

In [9]:
approximate_average = (normalized_prices[-1] - normalized_prices[0]) / (n - 1)
(approximate_average[..., None]*pw*b).flatten()

NameError: name 'normalized_prices' is not defined

In [9]:
get_normalized_prices(prices, b)

array([[20., 20.],
       [10., 10.],
       [ 2.,  2.],
       [ 1.,  1.]])

In [12]:
a = get_normalized_prices(prices, b)
get_average_daily_returns_partition_tecnalia(a, pw, 1)

array([-6.33333333, -3.16666667, -1.58333333, -0.79166667, -0.39583333,
       -6.33333333, -3.16666667, -1.58333333, -0.79166667, -0.39583333])

In [32]:
np.testing.assert_equal(get_qubo_covariance(npp), np.cov(selection.npp.T))

np_ = get_normalized_prices(prices, b)
a_ = get_average_daily_returns_partition_tecnalia(np_, pw)
np.testing.assert_almost_equal(get_qubo_returns(a_), np.diag(selection.expected_returns), decimal=15)

In [36]:
old = np.outer(selection.npp_last, selection.npp_last)

n, m = prices.shape
pw = get_partitions_granularity(w)
np_ = get_normalized_prices(prices, b)
npp = get_normalized_prices_partition(np_, pw)

partitions_granularity_broadcast = get_partitions_granularity_broadcast(pw, m)
new =  get_qubo_prices_quadratic(partitions_granularity_broadcast, b)

np.testing.assert_equal(new, old)

In [38]:
old = 2 * b * np.diag(selection.npp_last)
new = get_qubo_prices_linear(partitions_granularity_broadcast, b)
np.testing.assert_equal(new, old)

In [49]:
old_er = selection.expected_returns
new_er = get_average_daily_returns_partition_tecnalia(np_, pw)
np.testing.assert_almost_equal(old_er, new_er, decimal=15)

In [52]:
theta1 = 0.1
theta2 = 0.1
theta3 = 0.1

old_qubo_covariance = np.cov(selection.npp.T)  # (p, p)

# Set the Qubo values
old_qubo_returns = np.diag(selection.expected_returns)  # (p, p)
old_qubo_prices_linear = (
    2.0 * selection.b * np.diag(selection.npp_last)
)  # (p, p)
old_qubo_prices_quadratic = np.outer(
    selection.npp_last, selection.npp_last
)  # (p, p)

# Final QUBO formation, with bias and penalty values included
old_qi = (
    -theta1 * old_qubo_returns - theta2 * old_qubo_prices_linear
)  # (p, p).  eq (21a)
old_qij = (
    theta2 * old_qubo_prices_quadratic + theta3 * old_qubo_covariance
)  # (p, p). eq (21b)
old_qubo = typing.cast(npt.NDArray[np.floating[typing.Any]], qi + qij)

old_qubo_matrix = get_upper_triangular(old_qubo)
old_qubo_dict = get_qubo_dict(old_qubo_matrix)

In [60]:
n, m = prices.shape

# Compute the granularity partition
pw = get_partitions_granularity(w)

# Compute the  partition of the normalized prices
np_ = get_normalized_prices(prices, b)
npp = get_normalized_prices_partition(np_, pw)

# Compute the partitions of the average daily returns.
# adr = get_average_daily_returns(prices)
# adrp = get_average_daily_returns_partition(adr, pw)

adrp = get_average_daily_returns_partition_tecnalia(np_, pw)

# Set qubo values
qubo_covariance = get_qubo_covariance(npp)
qubo_returns = get_qubo_returns(adrp)
partitions_granularity_broadcast = get_partitions_granularity_broadcast(pw, m)
qubo_linear = get_qubo_prices_linear(partitions_granularity_broadcast, b)
qubo_quadratic = get_qubo_prices_quadratic(partitions_granularity_broadcast, b)

# Create qubo.
qi = -theta1 * qubo_returns - theta2 * qubo_linear  # (p, p).  eq (21a)
qij = theta2 * qubo_quadratic + theta3 * qubo_covariance  # (p, p). eq (21b)
qubo_ = typing.cast(Array, qi + qij)
qubo_matrix = get_upper_triangular(qubo_)
qubo_dict = get_qubo_dict(qubo_matrix)

In [61]:
np.testing.assert_equal(qubo_covariance, old_qubo_covariance)

In [62]:
np.testing.assert_equal(qubo_covariance, old_qubo_covariance)

In [65]:
np.testing.assert_almost_equal(qubo_returns, old_qubo_returns)

In [69]:
np.testing.assert_almost_equal(qubo_matrix, old_qubo_matrix)

In [3]:
from collections import Counter

c = Counter()
for l in [[1, 2, 3], [1, 2, 4]]:
    c.update(l)
c

Counter({1: 2, 2: 2, 3: 1, 4: 1})